In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import DatasetFolder
from torchvision.transforms import transforms
import random

# 全局参数
batch_size = 12
learning_rate = 0.001
num_epochs = 10

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 加载数据集
dataset = DatasetFolder('./MO_106/', loader=torchvision.datasets.folder.default_loader,
                        extensions=('jpg', 'jpeg', 'png'), transform=transform)

# 划分数据集
train_ratio = 0.7
valid_ratio = 0.2
test_ratio = 0.1

train_size = int(train_ratio * len(dataset))
valid_size = int(valid_ratio * len(dataset))
test_size = len(dataset) - train_size - valid_size

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# 定义模型、损失函数和优化器
model = torch.hub.load('AdeelH/pytorch-fpn', 'make_fpn_efficientnet',
                       force_reload=True, name='efficientnet_b5',
                       num_classes=106, fpn_channels=256, in_channels=3)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        
        # 将标签转换为One-Hot编码
        labels_onehot = nn.functional.one_hot(labels, num_classes=106).float()
        
        loss = criterion(outputs, labels_onehot)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    # 在验证集上验证模型性能
    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels_onehot)
            
            # 将One-Hot编码的标签转换回原始标签形式
            _, predicted = outputs.max(1)
            labels_original = torch.tensor([pred.item() for pred in predicted])
            
            valid_loss += loss.item()
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    valid_loss = valid_loss / len(valid_loader)
    valid_acc = correct / total

    # 打印训练过程中的损失和准确率
    print(f"Epoch {epoch+1}/{num_epochs}: "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

# 在测试集上进行推理
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        
        # 将One-Hot编码的标签转换回原始标签形式
        _, predicted = outputs.max(1)
        labels_original = torch.tensor([pred.item() for pred in predicted])
        
        loss = criterion(outputs, labels_onehot)
        test_loss += loss.item()
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_loss = test_loss / len(test_loader)
test_acc = correct / total

print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")


Downloading: "https://github.com/AdeelH/pytorch-fpn/zipball/master" to /Users/albert/.cache/torch/hub/master.zip
Using cache found in /Users/albert/.cache/torch/hub/lukemelas_EfficientNet-PyTorch_master


Loaded pretrained weights for efficientnet-b5


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 2